In [1]:
! pip install -qU memorizz yahooquery

In [2]:
import os
import sys

# Add the project root to the Python path
project_root = os.path.abspath(os.path.join(os.getcwd(), "..", "..", ".."))
sys.path.insert(0, project_root)


In [3]:
import getpass
import os

# Function to securely get and set environment variables
def set_env_securely(var_name, prompt):
    value = getpass.getpass(prompt)
    os.environ[var_name] = value

In [4]:
set_env_securely("MONGODB_URI", "Enter your MongoDB URI: ")

In [ ]:
set_env_securely("OPENAI_API_KEY", "Enter your OpenAI API key: ")

### Step 1: Initalize a Memory Provider

A Memory Provider is a core abstraction layer that manages the persistence, organization, and retrieval of all memory components within an agentic system. It serves as the central nervous system for memory management, providing standardized interfaces between AI agents and underlying storage technologies.


In [5]:
from src.memorizz.memory_provider.mongodb.provider import MongoDBConfig, MongoDBProvider

# Create a memory provider
mongodb_config = MongoDBConfig(uri=os.environ["MONGODB_URI"])
memory_provider = MongoDBProvider(mongodb_config)

In [6]:
import requests

def get_weather(latitude, longitude):
    response = requests.get(f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current=temperature_2m,wind_speed_10m&hourly=temperature_2m,relative_humidity_2m,wind_speed_10m")
    data = response.json()
    return data['current']['temperature_2m']

In [7]:
latitude = 40.7128
longitude = -74.0060
weather = get_weather(latitude, longitude)
print(weather)


22.7


In [8]:
from functools import lru_cache
from yahooquery import Ticker
import time

@lru_cache(maxsize=128)
def _fetch_price(symbol: str) -> float:
    """
    Internal helper to fetch the latest market price via yahooquery.
    Caching helps avoid repeated hits for the same symbol.
    """
    ticker = Ticker(symbol)
    # This returns a dict keyed by symbol:
    info = ticker.price or {}
    # regularMarketPrice holds the current trading price
    price = info.get(symbol.upper(), {}).get("regularMarketPrice")
    if price is None:
        raise ValueError(f"No price data for '{symbol}'")
    return price

def get_stock_price(
    symbol: str,
    currency: str = "USD",
    retry: int = 3,
    backoff: float = 0.5
) -> str:
    """
    Get the current stock price for a given symbol using yahooquery,
    with simple retry/backoff to handle occasional rate-limits.

    Parameters
    ----------
    symbol : str
        Stock ticker, e.g. "AAPL"
    currency : str, optional
        Currency code (Currently informational only; yahooquery returns native)
    retry : int, optional
        Number of retries on failure (default: 3)
    backoff : float, optional
        Backoff factor in seconds between retries (default: 0.5s)

    Returns
    -------
    str
        e.g. "The current price of AAPL is 172.34 USD."
    """
    symbol = symbol.upper()
    last_err = None
    for attempt in range(1, retry + 1):
        try:
            price = _fetch_price(symbol)
            return f"The current price of {symbol} is {price:.2f} {currency.upper()}."
        except Exception as e:
            last_err = e
            # simple backoff
            time.sleep(backoff * attempt)
    # if we get here, all retries failed
    raise RuntimeError(f"Failed to fetch price for '{symbol}' after {retry} attempts: {last_err}")


In [9]:
print(get_stock_price("AAPL"))

The current price of AAPL is 201.08 USD.


In [10]:
from src.memorizz import Toolbox
# Create a Toolbox instance
weather_toolbox = Toolbox(memory_provider=memory_provider)
stock_toolbox = Toolbox(memory_provider=memory_provider)
# Register the functions with the Toolbox
# These tools are now stored in the `ToolBox` store within the storage provider
weather_toolbox.register_tool(get_weather)
stock_toolbox.register_tool(get_stock_price)

'686016436724e210da3d0b6f'

In [11]:
from src.memorizz import MemAgent

# Create specialized agents
weather_agent = MemAgent(
    memory_provider=memory_provider,
    instruction="You are a weather specialist focused on weather data gathering and analysis."
)
weather_agent.add_tool(toolbox=weather_toolbox, persist=True)

weather_agent.save()

INFO:src.memorizz.memagent:Loading 1 available tools from toolbox (skipping database-only tools)
INFO:src.memorizz.memagent:Memagent None updated in the memory provider
INFO:src.memorizz.memagent:Memagent 6860164a6724e210da3d0b70 saved in the memory provider
INFO:src.memorizz.memagent:{
    "model": null,
    "tools": [
        {
            "_id": "6860163f6724e210da3d0b6e",
            "name": "get_weather",
            "description": "Retrieve the current weather information for a specific location identified by latitude and longitude coordinates. This function accesses weather data from a reliable source and provides insights into temperature, humidity, wind speed, and other meteorological conditions.",
            "parameters": [
                {
                    "name": "latitude",
                    "description": "The geographical latitude of the location. It should be a decimal number ranging from -90.0 to 90.0, with positive values indicating north of the equator and neg

MemAgent(agent_id=6860164a6724e210da3d0b70, memory_provider=<src.memorizz.memory_provider.mongodb.provider.MongoDBProvider object at 0x7fb9a8605880>)

In [12]:
stock_agent = MemAgent(
    memory_provider=memory_provider,
    instruction="You are a stock specialist focused on stock data gathering and analysis."
)
stock_agent.add_tool(toolbox=stock_toolbox)
stock_agent.save()

INFO:src.memorizz.memagent:Loading 1 available tools from toolbox (skipping database-only tools)
INFO:src.memorizz.memagent:Memagent None updated in the memory provider
INFO:src.memorizz.memagent:Memagent 6860164a6724e210da3d0b71 saved in the memory provider
INFO:src.memorizz.memagent:{
    "model": null,
    "tools": [
        {
            "_id": "686016436724e210da3d0b6f",
            "name": "get_stock_price",
            "description": "Get the current stock price for a specified stock symbol using the yahooquery API. This function incorporates a retry mechanism with exponential backoff to handle occasional rate-limit issues encountered during API calls. It captures and returns the latest stock price as a formatted string.",
            "parameters": [
                {
                    "name": "symbol",
                    "description": "String representing the stock ticker symbol. This parameter is required and is used to identify the stock for which the price is being fetch

MemAgent(agent_id=6860164a6724e210da3d0b71, memory_provider=<src.memorizz.memory_provider.mongodb.provider.MongoDBProvider object at 0x7fb9a8605880>)

In [13]:
# Create root agent with delegates
coordinator = MemAgent(
    memory_provider=memory_provider,
    delegates=[weather_agent, stock_agent],
    instruction="You coordinate complex tasks by delegating to specialized agents."
)

coordinator.save()

INFO:src.memorizz.memagent:Memagent 6860164a6724e210da3d0b72 saved in the memory provider
INFO:src.memorizz.memagent:{
    "model": null,
    "tools": [
        {
            "_id": "6860163f6724e210da3d0b6e",
            "name": "get_weather",
            "description": "Retrieve the current weather information for a specific location identified by latitude and longitude coordinates. This function accesses weather data from a reliable source and provides insights into temperature, humidity, wind speed, and other meteorological conditions.",
            "parameters": [
                {
                    "name": "latitude",
                    "description": "The geographical latitude of the location. It should be a decimal number ranging from -90.0 to 90.0, with positive values indicating north of the equator and negative values south.",
                    "type": "number",
                    "required": true
                },
                {
                    "name": "longit

MemAgent(agent_id=6860164b6724e210da3d0b74, memory_provider=<src.memorizz.memory_provider.mongodb.provider.MongoDBProvider object at 0x7fb9a8605880>)

In [14]:
# Execute multi-agent workflow
result = coordinator.run("Give me the weather in San Francisco and then the stock price of Apple")

INFO:src.memorizz.memagent:AGENT RUN START: Agent 6860164b6724e210da3d0b74 executing query: Give me the weather in San Francisco and then the ...
INFO:src.memorizz.memagent:AGENT RUN DEBUG: Agent has memory_ids: []
INFO:src.memorizz.memagent:Initialized orchestrator for agent 6860164b6724e210da3d0b74 with 2 delegates
INFO:src.memorizz.multi_agent_orchestrator:Starting multi-agent workflow for query: Give me the weather in San Francisco and then the stock price of Apple
INFO:src.memorizz.multi_agent_orchestrator:Root agent ID: 6860164b6724e210da3d0b74
INFO:src.memorizz.multi_agent_orchestrator:Number of delegates: 2
INFO:src.memorizz.multi_agent_orchestrator:Delegate IDs: ['6860164a6724e210da3d0b72', '6860164a6724e210da3d0b73']
INFO:src.memorizz.multi_agent_orchestrator:No existing session found, creating new root-level shared session
INFO:src.memorizz.multi_agent_orchestrator:Created new shared memory session: 6860164b6724e210da3d0b76
INFO:src.memorizz.multi_agent_orchestrator:Initial 

In [15]:
result

"Here is the consolidated response to your query:\n\n- The current weather in San Francisco (latitude: 37.7749, longitude: -122.4194) is 16.8°C.\n- The current stock price for Apple (AAPL) is $201.08 USD.\n\nIf you need more detailed weather information (such as humidity or wind speed) or more details about Apple's stock performance (such as recent price history), please let me know!\n\nThere are no conflicting results, and all aspects of your request have been addressed. No additional steps are needed unless you require further details."